In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import numpy as np
from math import log2
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList

In [2]:
# tensorflow version: 2.6.3, tfdv version: 1.3.0
print('tensorflow version: %s' % tf.__version__)
print('tfdv version: %s' % tfdv.__version__)

tensorflow version: 2.6.3
tfdv version: 1.3.0


In [3]:
DATA_PATH = './data/adult.data'

In [4]:
# Create noise rows for testing.
noise_rows = pd.DataFrame([
    {
        'age': 46, 
        'fnlwgt': 257473, 
        'education': 'Bachelors', 
        'education-num': 8,
        'marital-status': 'Married-civ-spouse', 
        'occupation': 'Plumber', 
        'relationship': 'Husband', 
        'race': 'Other', 
        'sex': 'Male',
        'capital-gain': 1000, 
        'capital-loss': 0, 
        'hours-per-week': 41, 
        'native-country': 'Australia',
        'label': '>50K'
    },
#     {
#         'age': 1000, 
#         'workclass': 'Private', 
#         'fnlwgt': 257473, 
#         'education': 'Masters', 
#         'education-num': 8,
#         'marital-status': 'Married-civ-spouse', 
#         'occupation': 'Prof-specialty', 
#         'relationship': 'Husband', 
#         'race': 'Black', 
#         'sex': 'Male',
#         'capital-gain': 0, 
#         'capital-loss': 0, 
#         'hours-per-week': 20, 
#         'native-country': 'Cameroon',
#         'label': '<=50K'
#     },
])

In [5]:
# Split data and add noises
df = pd.read_csv(DATA_PATH, skipinitialspace=True)
train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=False)
eval_df = pd.concat([eval_df, noise_rows]).reset_index(drop=True)
print(train_df.shape, eval_df.shape)

(26048, 15) (6514, 15)


In [6]:
print(df.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'label'],
      dtype='object')


In [7]:
train_stats = tfdv.generate_statistics_from_dataframe(train_df)
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

In [8]:
tfdv.visualize_statistics(
    lhs_statistics=eval_stats,  # left hand side
    rhs_statistics=train_stats,  # right hand side
    lhs_name='EVAL_DATASET',
    rhs_name='TRAIN_DATASET'
)

In [9]:
schema = tfdv.infer_schema(train_stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'workclass',STRING,required,,'workclass'
'fnlwgt',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'marital-status',STRING,required,,'marital-status'
'occupation',STRING,required,,'occupation'
'relationship',STRING,required,,'relationship'
'race',STRING,required,,'race'


,Values
Domain,
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'marital-status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'sex',"'Female', 'Male'"
'native-country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'label',"'<=50K', '>50K'"


In [10]:
# feature 확인
dataset_idx = 0

type_dict = {0: 'INT', 1: 'FLOAT', 2: 'STRING', 3: 'BYTES', 4: 'STRUCT'}
train_datasets = train_stats.datasets[dataset_idx]

feat_dict = {'INT': {}, 'FLOAT': {}, 'STRING': {}, 'BYTES': {}, 'STRUCT': {}}
for i in range(len(train_datasets.features)):
    feat_type = type_dict[train_datasets.features[i].type]
    feat_name = train_datasets.features[i].path.step[0]
    print('idx: %s, type: %s, name: %s' % (i, feat_type, feat_name))
    feat_dict[feat_type][i] = feat_name

idx: 0, type: INT, name: age
idx: 1, type: STRING, name: workclass
idx: 2, type: INT, name: fnlwgt
idx: 3, type: STRING, name: education
idx: 4, type: INT, name: education-num
idx: 5, type: STRING, name: marital-status
idx: 6, type: STRING, name: occupation
idx: 7, type: STRING, name: relationship
idx: 8, type: STRING, name: race
idx: 9, type: STRING, name: sex
idx: 10, type: INT, name: capital-gain
idx: 11, type: INT, name: capital-loss
idx: 12, type: INT, name: hours-per-week
idx: 13, type: STRING, name: native-country
idx: 14, type: STRING, name: label


In [11]:
for item in feat_dict['STRING'].values():
    tfdv.get_feature(schema, item).skew_comparator.infinity_norm.threshold = 0.00000000000001

for item in feat_dict['INT'].values():
    tfdv.get_feature(schema, item).skew_comparator.jensen_shannon_divergence.threshold = 0.00000000000001

# skew_comparator는 input statistics와 serving_statistics를 비교 
skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          serving_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'sex',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.000139949 (up to six significant digits), above the threshold 1e-14. The feature value with maximum difference is: Male"
'hours-per-week',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.000320346 (up to six significant digits), above the threshold 1e-14."
'marital-status',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.00976113 (up to six significant digits), above the threshold 1e-14. The feature value with maximum difference is: Married-civ-spouse"
'occupation',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.0119337 (up to six significant digits), above the threshold 1e-14. The feature value with maximum difference is: Adm-clerical"
'education-num',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.00022934 (up to six significant digits), above the threshold 1e-14."
'capital-gain',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.000171063 (up to six significant digits), above the threshold 1e-14."
'label',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.00618219 (up to six significant digits), above the threshold 1e-14. The feature value with maximum difference is: >50K"
'age',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 0.000781631 (up to six significant digits), above the threshold 1e-14."
'education',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.00685412 (up to six significant digits), above the threshold 1e-14. The feature value with maximum difference is: HS-grad"


In [12]:
# 위 예시는 2 데이터 셋 사이의 skew만을 볼 때 쓰는 것
# schema를 기반으로 target_stats의 anomalies를 보려면 아래처럼 serving_statistics를 빼고 target을 eval_stats으로
anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'native-country',Unexpected string values,Examples contain values missing from the schema: Australia (<1%).
'workclass',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.999846"
'occupation',Unexpected string values,Examples contain values missing from the schema: Plumber (<1%).


In [13]:
# Save results of anomalies.
# tfdv.validate_statistics() 할 때마다 index 바뀌어서 일단 아래처럼 저장
tfdv_result_dict = {}
for key in range(len(train_stats.datasets[0].features)):
    feat_name = skew_anomalies.drift_skew_info[key].path.step[0]
    skew_value = skew_anomalies.drift_skew_info[key].skew_measurements[0].value
    tfdv_result_dict[feat_name] = skew_value
print(tfdv_result_dict)

{'age': 0.0007816311306276795, 'capital-gain': 0.0001710629424013882, 'capital-loss': 4.7614878072574796e-05, 'education': 0.006854118402322251, 'education-num': 0.00022933953778635273, 'fnlwgt': 0.031201520323425255, 'hours-per-week': 0.0003203461998452674, 'label': 0.0061821933895544745, 'marital-status': 0.00976112968552334, 'native-country': 0.0037293753616289838, 'occupation': 0.011933732410781842, 'race': 0.0058659566354530845, 'relationship': 0.014279370118716134, 'sex': 0.00013994852515730738, 'workclass': 0.007531778957677093}


## L-Infinity Distance & Jensen-Shannon Divergence

TFDV에서 skew/drift를 판별할 때 L-Infinity Distance와 Jensen-Shannon Divergence를 사용함
- Categorical feature: L-Infinity Distance
- Numeric Feature: Jensen-Shannon Divergence (Distance가 아닌 Divergence 값으로 쓰임)

### L-Infinity Distance

- Input
    - `features[feature_index].string_stats.rank_histogram`
        - Collections of (label, sample_count) buckets
        - e.g. buckets { label: "Private", sample_count: 18117.0 } { ... }
    - `features[feature_idx].string_stats.common_stats.num_non_missing`
        - Value to perform normalization

- Logic
    - Normalize rank_histogram with num_non_missing value of each feature 
        - sample_count / num_non_missing
    - Union two different rank_histograms 
        - To align length of histogram_1 and histogram_2
    - Calculate normalized difference between histogram_1 and histogram_2 
        - The normalized value of label that was not included in original histogram is calculated as 0
    - Find max values among normalized difference between two histograms (equal to L-Infinity Distance)
    
- Original TFDV Source Code
    - https://github.com/tensorflow/data-validation/blob/46c98c898ca1127655e31ff0fcf35136b5f57b57/tensorflow_data_validation/anomalies/metrics.cc
        - GetLInftyNorm()
        - LInftyDistance()

In [14]:
# helper functions
def get_normalized_label_list_counts(datasets, feature_idx):
    label_len = len(datasets.features[feature_idx].string_stats.rank_histogram.buckets)
    feature_num_non_missing = datasets.features[feature_idx].string_stats.common_stats.num_non_missing
    label_list = []
    normalized_count_dict = {}
    # num_non_missing 으로 normalize
    for i in range(label_len):
        label = datasets.features[feature_idx].string_stats.rank_histogram.buckets[i].label
        count = datasets.features[feature_idx].string_stats.rank_histogram.buckets[i].sample_count
        label_list.append(label)
        normalized_count_dict[label] = count / feature_num_non_missing
    
    return label_list, normalized_count_dict

In [15]:
# 합집합으로 만들어보자
def calculate_l_infinity_dist(current_stats, target_stats, dataset_idx, feature_idx):
    current_datasets = current_stats.datasets[dataset_idx]
    target_datasets = target_stats.datasets[dataset_idx]
    
    current_label_list, current_norm_count_dict = get_normalized_label_list_counts(current_datasets, feature_idx)
    target_label_list, target_norm_count_dict = get_normalized_label_list_counts(target_datasets, feature_idx)

    # dataset마다 포함한 item이 다를 수 있어 합집합
    union_label_list = set(current_label_list) | set(target_label_list)

    # 차이 계산
    # 다만, 합집합으로 계산하고 없는 값은 일단 0으로 넣어서 차이 계산
    res_dict = {}
    for label in union_label_list:
        res_dict[label] = abs(current_norm_count_dict.get(label, 0) - target_norm_count_dict.get(label, 0))
    
    return max(res_dict.values())

In [16]:
feat_dict['STRING'].items()

dict_items([(1, 'workclass'), (3, 'education'), (5, 'marital-status'), (6, 'occupation'), (7, 'relationship'), (8, 'race'), (9, 'sex'), (13, 'native-country'), (14, 'label')])

In [17]:
# c++에서 double을 써서 metrics를 구현했는데 double은 15자리까지 정확히 표시하므로 아래와 같이 비교
for key, item in feat_dict['STRING'].items():
    l_infinity_dist_tfdv = round(tfdv_result_dict[item], 14)
    l_infinity_dist_cust = round(calculate_l_infinity_dist(train_stats, eval_stats, 0, key), 14)
    print('%s: %s(tfdv), %s(custom)' % (item, l_infinity_dist_tfdv, l_infinity_dist_cust))
    try:
        assert(l_infinity_dist_tfdv == l_infinity_dist_cust)
    except:
        print('--> error')

workclass: 0.00753177895768(tfdv), 0.00753177895768(custom)
education: 0.00685411840232(tfdv), 0.00685411840232(custom)
marital-status: 0.00976112968552(tfdv), 0.00976112968552(custom)
occupation: 0.01193373241078(tfdv), 0.01193373241078(custom)
relationship: 0.01427937011872(tfdv), 0.01427937011872(custom)
race: 0.00586595663545(tfdv), 0.00586595663545(custom)
sex: 0.00013994852516(tfdv), 0.00013994852516(custom)
native-country: 0.00372937536163(tfdv), 0.00372937536163(custom)
label: 0.00618219338955(tfdv), 0.00618219338955(custom)


### Jensen-Shannon Divergence

- Input
    - `features[feature_index].num_stats.histogram`
        - Collections of (low_value, high_value, sample_count) buckets.
        - e.g. buckets { low_value: 17.0, high_value: 24.3, sample_count: 4435.9744 } { ... }
        - Origianl length is 10. 
        - As two different histograms are likely to have different low_value and high_value, thus a modification of histogram through union is often necessary.
    - `features[feature_index].num_stats.common_stats.num_missing`
        - To check wheter to add nan value bucket

- Logic
    - Get union boundaries of two histograms using low_value and high_value.
        - Each low_value and high_value is treated as boundary.
        - If two histogram have different boundaries, these boundaries needed to be unioned in order to compare each bucket.
        - e.g. `[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]` union `[15, 25, 35, 45, 55, 65, 75, 85, 95, 105]` --> `[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105]`
    - Add values to newly created bucket. (Rebucketing)
        - Fill in empty buckets up to the first bucket in the existing histogram.
        - If existing bucket needed to be divided because of new boundary, distribute values based on uniform distribution.
            - `{low_value: 10, high_value: 20, sample_count: 100}` -> `{low_value: 10, high_value: 15, sample_count:50}, {low_value: 15, high_value: 20, sample_count: 50} `
        - Fill in empty buckets after the last bucket in the existing histogram.
    - Normalize rebucketted histogram sample_count. (Make it to 0.0 ~ 1.0)
    - Calculate through Jensen-Shannon Divergence formula.
        - Kullback-Leibler Divergence: `Dkl(p||q) = E[log(pi)-log(qi)] = sum(pi*log(pi/qi))`
        - Jensen-Shannon Divergence: `JSD(p||q) = 1/2*Dkl(p||m) + 1/2*Dkl(q||m) where m=(p+q)/2`
        - Summation of JSD values
    
- Original TFDV Source Code
    - https://github.com/tensorflow/data-validation/blob/46c98c898ca1127655e31ff0fcf35136b5f57b57/tensorflow_data_validation/anomalies/metrics.cc
        - UpdateJensenShannonDivergenceResult()

In [19]:
# Define calculation functions for Jensen-Shannon Divergence 
# https://hyeongminlee.github.io/post/prob002_kld_jsd/
# Jensen-shannon Divergence: Kullback-Leibler Divergence를 계량해 symmetric하게 거리 계산할 수 있게 만든 척도
# Kullback-Leibler Divergence: Dkl(p||q) = E[log(pi)-log(qi)] = sum(pi*log(pi/qi))
    # Where the “||” operator indicates “divergence” or Ps divergence from Q.(https://machinelearningmastery.com/divergence-between-probability-distributions/)
    # 위 식은 DKl(q||p)로 했을 때 같지 않아서 distance로 활용 불가
    # 반대로 했을 때도 적용되는 symetric이어야 distance metrics로 인정됨
# Jensen-shannon Divergence: JSD(p||q) = 1/2*Dkl(p||(p+q)/2) + 1/2*Dkl(q||(p+q)/2)
    # 이렇게 수식 바꿔주면 JSD(p||q) = JSD(q||p) 여서 distance로 활용 가능
    # 𝐽(𝑃,𝑄) = 1/2*(𝐷(𝑃∣∣𝑅)+𝐷(𝑄∣∣𝑅))와 같은 의미임. where 𝑅=1/2*(𝑃+𝑄) [symmetric하게 바꿔 준 것]
    # https://stats.stackexchange.com/questions/7630/clustering-should-i-use-the-jensen-shannon-divergence-or-its-square
    # square root를 tmaus Jensen-shannon distance인데 분포 간 서로 비교만 목적이고 거리 개념으로 환산하지 않을 것이면 그대로 JSD 써도 무방
# https://machinelearningmastery.com/divergence-between-probability-distributions/


# Calculate the kl divergence
def kl_divergence(p, q):
    res = 0
    for i in range(len(p)):
        if p[i] > 0 and q[i] > 0:
            res += p[i] * log2(p[i]/q[i])
    
    return res

# Calculate the js divergence
def js_divergence(p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

In [20]:
# Define distributions
# 10 buckets histogram example
p = np.asarray([0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10])
q = np.asarray([0.50, 0.10, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

# Calculate (P || Q)
kl_pq = kl_divergence(p, q)
print('KL(P || Q): %.3f bits' % kl_pq)
# Calculate (Q || P)
kl_qp = kl_divergence(q, p)
print('KL(Q || P): %.3f bits' % kl_qp)
print('')

# Calculate JS(P || Q)
js_pq = js_divergence(p, q)
print('JS(P || Q) divergence: %.3f bits' % js_pq)
print('JS(P || Q) distance: %.3f' % sqrt(js_pq))
# Calculate JS(Q || P)
js_qp = js_divergence(q, p)
print('JS(Q || P) divergence: %.3f bits' % js_qp)
print('JS(Q || P) distance: %.3f' % sqrt(js_qp))

KL(P || Q): 0.568 bits
KL(Q || P): 0.761 bits

JS(P || Q) divergence: 0.154 bits
JS(P || Q) distance: 0.392
JS(Q || P) divergence: 0.154 bits
JS(Q || P) distance: 0.392


In [21]:
feat_dict['INT']

{0: 'age',
 2: 'fnlwgt',
 4: 'education-num',
 10: 'capital-gain',
 11: 'capital-loss',
 12: 'hours-per-week'}

In [22]:
# Define helper functions
# 히스토그램 범위값이 안 맞는 상황 해결에서 이를 align시키기 위한 helper 함수 
# 운 좋게 히스토그램 범위 값이 같아서 바로 JS divergence를 구할 수도 있지만 그렇지 않다면 stats 데이터에 대한 전처리 필요

def get_initial_histogram(datasets, feature_index):
    histogram = {}
    histogram['buckets'] = []
    histogram['num_nan'] = datasets.features[feature_index].num_stats.common_stats.num_missing
    
    for i in range(10):
        low_value = datasets.features[feature_index].num_stats.histograms[0].buckets[i].low_value
        high_value = datasets.features[feature_index].num_stats.histograms[0].buckets[i].high_value
        sample_count = datasets.features[feature_index].num_stats.histograms[0].buckets[i].sample_count
        histogram['buckets'].append({'low_value': low_value, 'high_value': high_value, 'sample_count': sample_count})
    
    return histogram 


# 합집합으로 비교하려는 두 개의 low_value, high_value를 만들고, 경계를 산출
def get_union_boundaries(current_histogram, target_histogram):
    boundaries = set()
    for bucket in current_histogram['buckets']:
        boundaries.add(bucket['low_value'])
        boundaries.add(bucket['high_value'])
        
    for bucket in target_histogram['buckets']:
        boundaries.add(bucket['low_value'])
        boundaries.add(bucket['high_value'])
    
    boundaries = sorted(list(boundaries))
    
    return boundaries 


# 필요한 범위에 대해 추가된 바운더리만큼 히스토그램을 나눠서 분배
def add_buckets_to_histogram(bucket_boundaries, 
                            total_sample_count,
                            total_range_covered,
                            histogram):
    """
    e.g.
    # bucket 원래 값
    low_value: 19214.0, high_value: 165763.1, sample_count: 4158.425273604061
    # 비교 histogram 참조하면서 추가된 boundaries (중간에 156600.0 boundary가 추가되면서 히스토그램을 두 개로 쪼개야 되는 상황 생김)
    [19214.0, 156600.0, 165763.1]
    # uniform distribution을 가정해 (low_value - high_value) / cover_range 비율씩 sample_count를 분배
    # cover_range: 165763.1 - 19214.0
    {'low_value': 19214.0, 'high_value': 156600.0, 'sample_count': 3898.4163985951977}
    {'low_value': 156600.0, 'high_value': 165763.1, 'sample_count': 260.0088750088632}

    """
    
    num_new_buckets = len(bucket_boundaries) - 1  # boundaries는 buckets보다 1개 값이 더 있음
    for i in range(num_new_buckets):
        new_bucket = {}
        new_bucket['low_value'] = bucket_boundaries[i]
        new_bucket['high_value'] = bucket_boundaries[i+1]
        # uniform distribution으로 쪼개는 코드
        new_bucket['sample_count'] = ((bucket_boundaries[i+1] - bucket_boundaries[i]) / total_range_covered) * total_sample_count
        
        histogram['buckets'].append(new_bucket)
        

# boundaries에 맞게 각각 히스토그램을 rebucketing
def rebucket_histogram(boundaries, histogram):
    rebuck_hist = {}
    rebuck_hist['buckets'] = []
    rebuck_hist['num_nan'] = histogram['num_nan']
    
    index = 0
    max_index = len(boundaries) - 1

    for bucket in histogram['buckets']:
        low_value = bucket['low_value']
        high_value = bucket['high_value']
        sample_count = bucket['sample_count']
        
        # 원래 자신이 가지고 있던 값보다 작은 buket들 값 설정 (0으로 설정)
        while low_value > boundaries[index]:
            new_bucket = {}
            new_bucket['low_value'] = boundaries[index]
            index += 1
            new_bucket['high_value'] = boundaries[index]
            new_bucket['sample_count'] = 0.0

            rebuck_hist['buckets'].append(new_bucket)

        # 추가 예외 처리 부분인데 일단 비활성화
        # if low_value == high_value and low_value == boundaries[index]:
        #     new_bucket = {}
        #     new_bucket['low_value'] = boundaries[index]
        #     index += 1
        #     new_bucket['high_value'] = boundaries[index]
        #     new_bucket['sample_count'] = sample_count

        #     rebuck_hist.append(new_bucket)
        #     continue

        # 쪼개야 되는 범위 산출
        covered_boundaries = []
        while high_value > boundaries[index]:
            covered_boundaries.append(boundaries[index])
            index += 1
        covered_boundaries.append(boundaries[index])  # 같은 값일 때 자기 포함

        if len(covered_boundaries) > 0:
            add_buckets_to_histogram(covered_boundaries, sample_count, high_value - low_value, rebuck_hist)
    
    # 원래 자신의 범위 값 넘어선 범위들 bucket에 대한 값 설정 (0으로 설정)
    for i in range(index, max_index):
        new_bucket = {}
        new_bucket['low_value'] = boundaries[index]
        new_bucket['high_value'] = boundaries[index + 1]
        new_bucket['sample_count'] = 0.0

        rebuck_hist['buckets'].append(new_bucket)
    return rebuck_hist


def align_histogram(boundaries, current_histogram, target_histogram):
    boundaries = get_union_boundaries(current_histogram, target_histogram)
    current_histogram_rebucketted = rebucket_histogram(boundaries, current_histogram)
    target_histogram_rebucketted = rebucket_histogram(boundaries, target_histogram)
    return current_histogram_rebucketted, target_histogram_rebucketted


In [23]:
# tfdv c++ 코드처럼 조작하기 위한 histogram 정보 불러오기 및 boundaries 산출
# boundaries 원래 11개 값으로 10개 히스토그램 만들었던 것들이 2개 데이터를 합치는 과정에서 22개 boundaries와 21개 histogram으로 변함
train_datasets = train_stats.datasets[0]
target_datasets = eval_stats.datasets[0]

current_hist = get_initial_histogram(train_datasets, 2)
target_hist = get_initial_histogram(target_datasets, 2)
union_boundaries = get_union_boundaries(current_hist, target_hist)

print('# union_boundaries')
print(union_boundaries)
current_hist_rebuck, target_hist_rebuck = align_histogram(union_boundaries, current_hist, target_hist)
# boundaries - 1 만큼 나와야 함
print('# length of union boundaries: %s' % (len(union_boundaries)))
print('# current_hist_rebucket length: %s, target_hist_rebucket length: %s' 
      % (len(current_hist_rebuck['buckets']), len(target_hist_rebuck['buckets'])))

# union_boundaries
[12285.0, 13769.0, 124953.4, 159527.0, 236137.8, 306769.0, 347322.19999999995, 454011.0, 458506.6, 569691.0, 601253.0, 680875.3999999999, 748495.0, 792059.7999999999, 895737.0, 903244.2, 1014428.6, 1042979.0, 1125613.0, 1190221.0, 1337463.0, 1484705.0]
# length of union boundaries: 22
# current_hist_rebucket length: 21, target_hist_rebucket length: 21


In [24]:
# Check rebucketted histogram values.
# current_hist_rebuck

In [25]:
# Check rebucketted histogram values.
# target_hist_rebuck

In [26]:
def calculate_jensen_shannon_divergence(current_stats, target_stats, dataset_idx, feature_idx):
    current_datasets = current_stats.datasets[dataset_idx]   
    target_datasets = target_stats.datasets[dataset_idx]
    
    # tfdv c++ 코드처럼 조작하기 위한 histogram 정보 불러오기 및 boundaries 산출
    # boundaries 원래 11개 값으로 10개 히스토그램 만들었던 것들이 2개 데이터를 합치는 과정에서 22개 boundaries와 21개 histogram으로 변함
    current_hist = get_initial_histogram(train_datasets, feature_idx)
    target_hist = get_initial_histogram(target_datasets, feature_idx)
    union_boundaries = get_union_boundaries(current_hist, target_hist)
    current_hist_rebuck, target_hist_rebuck = align_histogram(union_boundaries, current_hist, target_hist)
    bucket_count = len(union_boundaries) - 1
    
    # nan bucket 추가 (tfdv가 이런 식으로 함, sort 안 되서 align 한 후에 해야 됨)
    if current_hist_rebuck['num_nan'] > 0 or target_hist_rebuck['num_nan'] > 0:
        current_hist_rebuck['buckets'].append({'low_value': None, 'high_value': None, 'sample_count': current_hist_rebuck['num_nan']})
        target_hist_rebuck['buckets'].append({'low_value': None, 'high_value': None, 'sample_count': target_hist_rebuck['num_nan']})
        bucket_count += 1
    
    current_num = 0
    for bucket in current_hist_rebuck['buckets']:
        current_num += bucket['sample_count']
    target_num = 0
    for bucket in target_hist_rebuck['buckets']:
        target_num += bucket['sample_count']
    
    current_hist_values = []
    for idx in range(bucket_count):
        sample_cnt = current_hist_rebuck['buckets'][idx]['sample_count']
        normalized_cnt = sample_cnt / current_num
        current_hist_values.append(normalized_cnt)
    target_hist_values = []
    for idx in range(bucket_count):
        sample_cnt = target_hist_rebuck['buckets'][idx]['sample_count']
        normalized_cnt = sample_cnt / target_num
        target_hist_values.append(normalized_cnt)
    
    # for i in range(len(current_hist_values)):
    #    print(i, current_hist_values[i], target_hist_values[i])
    
    return js_divergence(np.asarray(current_hist_values), np.asarray(target_hist_values))

In [27]:
# c++에서 double을 써서 metrics를 구현했는데 double은 15자리까지 정확히 표시하므로 아래와 같이 비교
for key, item in feat_dict['INT'].items():
    jsd_tfdv = round(tfdv_result_dict[item], 14)
    jsd_cust = round(calculate_jensen_shannon_divergence(train_stats, eval_stats, 0, key), 14)
    print('%s: %s(tfdv), %s(custom)' % (item, jsd_tfdv, jsd_cust))
    try:
        assert(jsd_tfdv == jsd_cust)
    except:
        print('--> error')

age: 0.00078163113063(tfdv), 0.00078163113063(custom)
fnlwgt: 0.03120152032343(tfdv), 0.03120152032343(custom)
education-num: 0.00022933953779(tfdv), 0.00022933953779(custom)
capital-gain: 0.0001710629424(tfdv), 0.0001710629424(custom)
capital-loss: 4.761487807e-05(tfdv), 4.761487807e-05(custom)
hours-per-week: 0.00032034619985(tfdv), 0.00032034619985(custom)
